<a href="https://colab.research.google.com/github/YoheiFukuhara/nlp100-2020/blob/main/08.%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88/70_%E5%8D%98%E8%AA%9E%E3%83%99%E3%82%AF%E3%83%88%E3%83%AB%E3%81%AE%E5%92%8C%E3%81%AB%E3%82%88%E3%82%8B%E7%89%B9%E5%BE%B4%E9%87%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例$x_i$の特徴ベクトル$\boldsymbol{x}_i$を並べた行列$X$と正解ラベルを並べた行列（ベクトル）$Y$を作成したい．

```math
X = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \dots \\ 
  \boldsymbol{x}_n \\ 
\end{pmatrix} \in \mathbb{R}^{n \times d},
Y = \begin{pmatrix} 
  y_1 \\ 
  y_2 \\ 
  \dots \\ 
  y_n \\ 
\end{pmatrix} \in \mathbb{N}^{n}
```

ここで，$n$は学習データの事例数であり，$\boldsymbol x_i \in \mathbb{R}^d$と$y_i \in \mathbb N$はそれぞれ，$i \in \{1, \dots, n\}$番目の事例の特徴量ベクトルと正解ラベルを表す．なお，今回は「ビジネス」「科学技術」「エンターテイメント」「健康」の4カテゴリ分類である．$\mathbb N_{<4}$で$4$未満の自然数（$0$を含む）を表すことにすれば，任意の事例の正解ラベル$y_i$は$y_i \in \mathbb N_{<4}$で表現できる．
以降では，ラベルの種類数を$L$で表す（今回の分類タスクでは$L=4$である）．

$i$番目の事例の特徴ベクトル$\boldsymbol x_i$は，次式で求める．

$$\boldsymbol x_i = \frac{1}{T_i} \sum_{t=1}^{T_i} \mathrm{emb}(w_{i,t})$$

ここで，$i$番目の事例は$T_i$個の（記事見出しの）単語列$(w_{i,1}, w_{i,2}, \dots, w_{i,T_i})$から構成され，$\mathrm{emb}(w) \in \mathbb{R}^d$は単語$w$に対応する単語ベクトル（次元数は$d$）である．すなわち，$i$番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものが$\boldsymbol x_i$である．今回は単語ベクトルとして，問題60でダウンロードしたものを用いればよい．$300$次元の単語ベクトルを用いたので，$d=300$である．
$i$番目の事例のラベル$y_i$は，次のように定義する．

```math
y_i = \begin{cases}
0 & (\mbox{記事}\boldsymbol x_i\mbox{が「ビジネス」カテゴリの場合}) \\
1 & (\mbox{記事}\boldsymbol x_i\mbox{が「科学技術」カテゴリの場合}) \\
2 & (\mbox{記事}\boldsymbol x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
3 & (\mbox{記事}\boldsymbol x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases}
```

なお，カテゴリ名とラベルの番号が一対一で対応付いていれば，上式の通りの対応付けでなくてもよい．

以上の仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．

 + 学習データの特徴量行列: $X_{\rm train} \in \mathbb{R}^{N_t \times d}$
 + 学習データのラベルベクトル: $Y_{\rm train} \in \mathbb{N}^{N_t}$
 + 検証データの特徴量行列: $X_{\rm valid} \in \mathbb{R}^{N_v \times d}$
 + 検証データのラベルベクトル: $Y_{\rm valid} \in \mathbb{N}^{N_v}$
 + 評価データの特徴量行列: $X_{\rm test} \in \mathbb{R}^{N_e \times d}$
 + 評価データのラベルベクトル: $Y_{\rm test} \in \mathbb{N}^{N_e}$

なお，$N_t, N_v, N_e$はそれぞれ，学習データの事例数，検証データの事例数，評価データの事例数である．

In [1]:
import string

import tensorflow as tf
import pandas as pd
import numpy as np
from google.colab import drive
from gensim.models import KeyedVectors

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!python --version
!pip show google pandas gensim tensorflow numpy

Python 3.7.12
Name: google
Version: 2.0.3
Summary: Python bindings to the Google search engine.
Home-page: http://breakingcode.wordpress.com/
Author: Mario Vilas
Author-email: mvilas@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: beautifulsoup4
Required-by: 
---
Name: pandas
Version: 1.1.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: None
Author-email: None
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: pytz, numpy, python-dateutil
Required-by: xarray, vega-datasets, statsmodels, sklearn-pandas, seaborn, pymc3, plotnine, pandas-profiling, pandas-gbq, pandas-datareader, mlxtend, mizani, holoviews, gspread-dataframe, google-colab, fix-yahoo-finance, fbprophet, fastai, cufflinks, cmdstanpy, arviz, altair
---
Name: gensim
Version: 3.6.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radi

In [3]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/'
BASE_PATH08 = BASE_PATH + '08.NeuralNetworks/'

In [4]:
%%time
w2v_model = KeyedVectors.load_word2vec_format(BASE_PATH+'07.WordVector/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

CPU times: user 2min 23s, sys: 5.71 s, total: 2min 28s
Wall time: 2min 33s


In [5]:
# 同じ長さでないとエラーがでるので第2引数の長さ調整(置換元と置換先文字は1:1だから)
table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
def vectorize(title):
    tokens = title.translate(table).split()
    vectors = [w2v_model[token] for token in tokens if token in w2v_model]
    
    # DataFrameの要素にNumpy Arrayを格納
    vectors = np.array(sum(vectors)/len(vectors))
    return vectors

In [6]:
def make_example(title, category):
    features={
       'title' : tf.train.Feature(bytes_list=tf.train.BytesList(value=[title])),
       'category' : tf.train.Feature(int64_list=tf.train.Int64List(value=[category]))
    }
    return tf.train.Example(features=tf.train.Features(feature=features))

In [7]:
def process_data(type_):
    df = pd.read_table(BASE_PATH08+'input/'+type_+'.txt')
    titles = df['title'].map(vectorize).explode().astype('float32')
    titles = titles.values.reshape(len(df), 300)
    categories = df['category'].replace({'b':0, 't':1, 'e':2, 'm':3})

    with tf.io.TFRecordWriter(BASE_PATH08+type_+'.tfrecord') as writer:
        for i, (title, category) in enumerate(zip(titles, categories)):
            ex = make_example(title.tobytes(), category)
            writer.write(ex.SerializeToString())

In [8]:
%%time
process_data('train')
process_data('valid')
process_data('test')

CPU times: user 2.28 s, sys: 238 ms, total: 2.52 s
Wall time: 4.34 s


In [17]:
!find /content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/08.NeuralNetworks -name '*.tfrecord' -ls

       77   1640 -rw-------   1 root     root      1679352 Sep 23 01:09 /content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/08.NeuralNetworks/valid.tfrecord
       82   1640 -rw-------   1 root     root      1679352 Sep 23 01:09 /content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/08.NeuralNetworks/test.tfrecord
       72  13116 -rw-------   1 root     root     13429788 Sep 23 01:09 /content/drive/MyDrive/ColabNotebooks/ML/NLP100_2020/08.NeuralNetworks/train.tfrecord
